In [1]:
import pandas as pd
import os
import requests as re
import datetime
import time
from IPython.display import JSON
import json

# Import Dataframes

### Flights

In [2]:
pd.set_option('max_columns', None)
#pd.reset_option('max_columns')

In [3]:
# Import df_flights dataframes. Specify datatype of cancellation_code so there are not mixed datatypes present.
# We will use statistics from the last week of December 2018 to help predict the first week of January 2019.

# df_flights_Dec_2018:   Dec 24 - 31, 2018
# df_flights_Dec_2019:   Dec 24 - 31, 2019
# df_flights:            Jan 1 - 8, 2019 (after filtering)
# df_flights_test:       Jan 1 - 8, 2020

df_flights_Dec_2018 = pd.read_csv('flights_Dec_2018.csv', dtype={'cancellation_code':'object'})
df_flights_Dec_2019 = pd.read_csv('flights_Dec_2019.csv', dtype={'cancellation_code':'object'})
df_flights = pd.read_csv('flights_data.csv', dtype={'cancellation_code':'object'})

# Filter df_flights to exclude January 2018
df_flights = df_flights[df_flights['fl_date'] >= '2019-01-01']

In [4]:
# # Check flight delay bins
# print(pd.cut(df_flights_Dec_2018['dep_delay'], bins=10).value_counts())
# print(pd.cut(df_flights_Dec_2019['dep_delay'], bins=10).value_counts())
# print(pd.cut(df_flights['dep_delay'], bins=10).value_counts())

In [5]:
# Filter out extreme delays which account for less than 1% of flights, for more accurate stats/modelling
df_flights_Dec_2018 = df_flights_Dec_2018[df_flights_Dec_2018['dep_delay'] < 325.8]
df_flights_Dec_2019 = df_flights_Dec_2019[df_flights_Dec_2019['dep_delay'] < 305.8]
df_flights = df_flights[df_flights['dep_delay'] < 300.4]

In [6]:
df_flights_Dec_2018.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-12-31,F9,F9,F9,1753,F9,N211FR,1753,13204,MCO,"Orlando, FL",11109,COS,"Colorado Springs, CO",2115,2105.0,-10.0,15.0,2120.0,2331.0,33.0,2321,4.0,43.0,0.0,NaN,0.0,N,246.0,299.0,251.0,1.0,1520.0,0.0,0.0,43.0,0.0,0.0,NaN,NaN,NaN,NaN


### Flights_test

In [7]:
# Look through flight data from January 2020
df_flights_test = pd.read_csv('flights_test.csv')

# Move column names to first row of data
df_flights_test = (df_flights_test.T.reset_index().T.reset_index(drop=True))

# Define new column names
df_flights_test.columns = ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance']

# Clean up fl_date
df_flights_test['fl_date'] = df_flights_test['fl_date'].str[0:10]

In [8]:
df_flights_test.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN.1,WN.2,5888,WN.3,N951WN,5888.1,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363


# Feature Engineering

In [11]:
# Get hour of departure and arrival times (approximate using crs for 2020)
df_flights_Dec_2018['hour_of_day_dep'] = df_flights_Dec_2018['dep_time'] // 100
df_flights_Dec_2018['hour_of_day_arr'] = df_flights_Dec_2018['arr_time'] // 100
df_flights_Dec_2019['hour_of_day_dep'] = df_flights_Dec_2019['dep_time'] // 100
df_flights_Dec_2019['hour_of_day_arr'] = df_flights_Dec_2019['arr_time'] // 100
df_flights['hour_of_day_dep'] = df_flights['dep_time'] // 100
df_flights['hour_of_day_arr'] = df_flights['arr_time'] // 100
df_flights_test['hour_of_day_dep'] = df_flights_test['crs_dep_time'].astype(int) // 100
df_flights_test['hour_of_day_arr'] = df_flights_test['crs_arr_time'].astype(int) // 100

# Set delay nulls to zero (zero minutes of delay instead of null)
df_flights_Dec_2018[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flights_Dec_2018[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(value=0)
df_flights_Dec_2019[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flights_Dec_2019[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(value=0)
df_flights[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flights[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(value=0)

# Exclude diverted and cancelled flights
df_flights_Dec_2018 = df_flights_Dec_2018[df_flights_Dec_2018['diverted'] != 1]
df_flights_Dec_2018 = df_flights_Dec_2018[df_flights_Dec_2018['cancelled'] != 1]
df_flights_Dec_2019 = df_flights_Dec_2019[df_flights_Dec_2019['diverted'] != 1]
df_flights_Dec_2019 = df_flights_Dec_2019[df_flights_Dec_2019['cancelled'] != 1]
df_flights = df_flights[df_flights['diverted'] != 1]
df_flights = df_flights[df_flights['cancelled'] != 1]

# Format fl_date column as a date
df_flights_Dec_2018['fl_date'] = pd.to_datetime(df_flights_Dec_2018['fl_date'])
df_flights_Dec_2019['fl_date'] = pd.to_datetime(df_flights_Dec_2019['fl_date'])
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'])
df_flights_test['fl_date'] = pd.to_datetime(df_flights_test['fl_date'])

# Create 'state' column
df_flights_Dec_2018['state'] = df_flights_Dec_2018['origin_city_name'].str[-2:]
df_flights_Dec_2019['state'] = df_flights_Dec_2019['origin_city_name'].str[-2:]
df_flights['state'] = df_flights['origin_city_name'].str[-2:]
df_flights_test['state'] = df_flights_test['origin_city_name'].str[-2:]

# Create fl_day column
df_flights_Dec_2018['fl_day'] = pd.DatetimeIndex(df_flights_Dec_2018['fl_date']).day
df_flights_Dec_2019['fl_day'] = pd.DatetimeIndex(df_flights_Dec_2019['fl_date']).day
df_flights['fl_day'] = pd.DatetimeIndex(df_flights['fl_date']).day
df_flights_test['fl_day'] = pd.DatetimeIndex(df_flights_test['fl_date']).day

# NOTE: statistics will be gathered from Dec 2018/2019, and applied to Jan 2019/2020 respectively to simulate "1 week in advance" predictions.
# Get and apply daily mean flight delay values
df_daily_delay_means_Dec_2018 = df_flights_Dec_2018.groupby('fl_day').mean()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights['daily_arr_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['arr_delay'][row['fl_day']-1], axis=1)
df_flights['daily_carrier_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['carrier_delay'][row['fl_day']-1], axis=1)
df_flights['daily_weather_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['weather_delay'][row['fl_day']-1], axis=1)
df_flights['daily_nas_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['nas_delay'][row['fl_day']-1], axis=1)
df_flights['daily_security_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['security_delay'][row['fl_day']-1], axis=1)
df_flights['daily_late_aircraft_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means_Dec_2018['late_aircraft_delay'][row['fl_day']-1], axis=1)

df_daily_delay_means_Dec_2019 = df_flights_Dec_2019.groupby('fl_day').mean()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights_test['daily_arr_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['arr_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_carrier_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['carrier_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_weather_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['weather_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_nas_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['nas_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_security_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['security_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_late_aircraft_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means_Dec_2019['late_aircraft_delay'][row['fl_day']-1], axis=1)

# Get and apply daily standard deviation of flight delay values
df_daily_delay_std_Dec_2018 = df_flights_Dec_2018.groupby('fl_day').std()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights['daily_arr_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['arr_delay'][row['fl_day']-1], axis=1)
df_flights['daily_carrier_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['carrier_delay'][row['fl_day']-1], axis=1)
df_flights['daily_weather_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['weather_delay'][row['fl_day']-1], axis=1)
df_flights['daily_nas_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['nas_delay'][row['fl_day']-1], axis=1)
df_flights['daily_security_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['security_delay'][row['fl_day']-1], axis=1)
df_flights['daily_late_aircraft_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std_Dec_2018['late_aircraft_delay'][row['fl_day']-1], axis=1)

df_daily_delay_std_Dec_2019 = df_flights_Dec_2019.groupby('fl_day').std()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights_test['daily_arr_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['arr_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_carrier_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['carrier_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_weather_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['weather_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_nas_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['nas_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_security_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['security_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_late_aircraft_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std_Dec_2019['late_aircraft_delay'][row['fl_day']-1], axis=1)

In [12]:
# Get and apply hourly mean flight delay values
dep_hourly_delay_mean_Dec_2018 = df_flights_Dec_2018.groupby('hour_of_day_dep').mean()['dep_delay']
arr_hourly_delay_mean_Dec_2018 = df_flights_Dec_2018.groupby('hour_of_day_arr').mean()['arr_delay']
dep_hourly_delay_mean_Dec_2019 = df_flights_Dec_2019.groupby('hour_of_day_dep').mean()['dep_delay']
arr_hourly_delay_mean_Dec_2019 = df_flights_Dec_2019.groupby('hour_of_day_arr').mean()['arr_delay']

df_flights['dep_mean_hourly_delay'] = df_flights.apply(lambda row: dep_hourly_delay_mean_Dec_2018[row['hour_of_day_dep']], axis=1)
df_flights['arr_mean_hourly_delay'] = df_flights.apply(lambda row: arr_hourly_delay_mean_Dec_2018[row['hour_of_day_arr']], axis=1)
df_flights_test['dep_mean_hourly_delay'] = df_flights_test.apply(lambda row: dep_hourly_delay_mean_Dec_2019[row['hour_of_day_dep']], axis=1)
df_flights_test['arr_mean_hourly_delay'] = df_flights_test.apply(lambda row: arr_hourly_delay_mean_Dec_2019[row['hour_of_day_arr']], axis=1)


# Get and apply hourly standard deviation of flight delay values
dep_hourly_delay_std_Dec_2018 = df_flights_Dec_2018.groupby('hour_of_day_dep').std()['dep_delay']
arr_hourly_delay_std_Dec_2018 = df_flights_Dec_2018.groupby('hour_of_day_arr').std()['arr_delay']
dep_hourly_delay_std_Dec_2019 = df_flights_Dec_2019.groupby('hour_of_day_dep').std()['dep_delay']
arr_hourly_delay_std_Dec_2019 = df_flights_Dec_2019.groupby('hour_of_day_arr').std()['arr_delay']

df_flights['dep_std_hourly_delay'] = df_flights.apply(lambda row: dep_hourly_delay_std_Dec_2018[row['hour_of_day_dep']], axis=1)
df_flights['arr_std_hourly_delay'] = df_flights.apply(lambda row: arr_hourly_delay_std_Dec_2018[row['hour_of_day_arr']], axis=1)
df_flights_test['dep_std_hourly_delay'] = df_flights_test.apply(lambda row: dep_hourly_delay_std_Dec_2019[row['hour_of_day_dep']], axis=1)
df_flights_test['arr_std_hourly_delay'] = df_flights_test.apply(lambda row: arr_hourly_delay_std_Dec_2019[row['hour_of_day_arr']], axis=1)

In [13]:
# Calculate and apply mean marketing (mkt) and operating (op) carrier delays
df_mean_mkt_carrier_delays_Dec_2018 = df_flights_Dec_2018.groupby('mkt_unique_carrier').mean()['carrier_delay']
df_mean_op_carrier_delays_Dec_2018 = df_flights_Dec_2018.groupby('op_unique_carrier').mean()['carrier_delay']
df_mean_mkt_carrier_delays_Dec_2019 = df_flights_Dec_2019.groupby('mkt_unique_carrier').mean()['carrier_delay']
df_mean_op_carrier_delays_Dec_2019 = df_flights_Dec_2019.groupby('op_unique_carrier').mean()['carrier_delay']

df_flights['mean_mkt_carrier_delay'] = df_flights.apply(lambda row: df_mean_mkt_carrier_delays_Dec_2018[row['mkt_unique_carrier']], axis=1)
df_flights['mean_op_carrier_delay'] = df_flights.apply(lambda row: df_mean_op_carrier_delays_Dec_2018[row['op_unique_carrier']], axis=1)
df_flights_test['mean_mkt_carrier_delay'] = df_flights_test.apply(lambda row: df_mean_mkt_carrier_delays_Dec_2019[row['mkt_unique_carrier']], axis=1)
df_flights_test['mean_op_carrier_delay'] = df_flights_test.apply(lambda row: df_mean_op_carrier_delays_Dec_2019[row['op_unique_carrier']] if row['op_unique_carrier'] in df_mean_op_carrier_delays_Dec_2019.index else df_mean_op_carrier_delays_Dec_2019.mean(), axis=1)


# Calculate and apply standard deviation of marketing (mkt) and operating (op) carrier delays
df_std_mkt_carrier_delays_Dec_2018 = df_flights_Dec_2018.groupby('mkt_unique_carrier').std()['carrier_delay']
df_std_op_carrier_delays_Dec_2018 = df_flights_Dec_2018.groupby('op_unique_carrier').std()['carrier_delay']
df_std_mkt_carrier_delays_Dec_2019 = df_flights_Dec_2019.groupby('mkt_unique_carrier').std()['carrier_delay']
df_std_op_carrier_delays_Dec_2019 = df_flights_Dec_2019.groupby('op_unique_carrier').std()['carrier_delay']

df_flights['std_mkt_carrier_delay'] = df_flights.apply(lambda row: df_std_mkt_carrier_delays_Dec_2018[row['mkt_unique_carrier']], axis=1)
df_flights['std_op_carrier_delay'] = df_flights.apply(lambda row: df_std_op_carrier_delays_Dec_2018[row['op_unique_carrier']], axis=1)
df_flights_test['std_mkt_carrier_delay'] = df_flights_test.apply(lambda row: df_std_mkt_carrier_delays_Dec_2019[row['mkt_unique_carrier']], axis=1)
df_flights_test['std_op_carrier_delay'] = df_flights_test.apply(lambda row: df_std_op_carrier_delays_Dec_2019[row['op_unique_carrier']] if row['op_unique_carrier'] in df_std_op_carrier_delays_Dec_2019.index else df_std_op_carrier_delays_Dec_2019.mean(), axis=1)

In [14]:
# Add relationship between tail number and delay time
tail_num_delay_means_Dec_2018 = df_flights_Dec_2018.groupby('tail_num').mean()['arr_delay']
tail_num_delay_std_Dec_2018 = df_flights_Dec_2018.groupby('tail_num').std()['arr_delay']
tail_num_delay_means_Dec_2019 = df_flights_Dec_2019.groupby('tail_num').mean()['arr_delay']
tail_num_delay_std_Dec_2019 = df_flights_Dec_2019.groupby('tail_num').std()['arr_delay']

df_flights['mean_tail_num_arr_delay'] = df_flights.apply(lambda row: tail_num_delay_means_Dec_2018[row['tail_num']] if row['tail_num'] in tail_num_delay_means_Dec_2018.index else tail_num_delay_means_Dec_2018.mean(), axis=1)
df_flights['std_tail_num_arr_delay'] = df_flights.apply(lambda row: tail_num_delay_std_Dec_2018[row['tail_num']] if row['tail_num'] in tail_num_delay_std_Dec_2018.index else tail_num_delay_std_Dec_2018.mean(), axis=1)
df_flights['std_tail_num_arr_delay'].fillna(value=tail_num_delay_std_Dec_2018.mean(),inplace=True)  # Fix null values

df_flights_test['mean_tail_num_arr_delay'] = df_flights_test.apply(lambda row: tail_num_delay_means_Dec_2019[row['tail_num']] if row['tail_num'] in tail_num_delay_means_Dec_2019.index else tail_num_delay_means_Dec_2019.mean(), axis=1)
df_flights_test['std_tail_num_arr_delay'] = df_flights_test.apply(lambda row: tail_num_delay_std_Dec_2019[row['tail_num']] if row['tail_num'] in tail_num_delay_std_Dec_2019.index else tail_num_delay_std_Dec_2019.mean(), axis=1)
df_flights_test['std_tail_num_arr_delay'].fillna(value=tail_num_delay_std_Dec_2019.mean(),inplace=True)  # Fix null values

In [26]:
df_flights;

In [ ]:
# One hot encoding?

# Dropping Columns, Construct Final DFs

In [15]:
# Drop inappropriate columns (this should be the final step, after other feature engineering is complete).
# Except for the arr_delay column, df_flights_final should have the same columns as df_flights_test in order to use both as training/testing X values.

df_flights_final = df_flights.drop(columns=['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'no_name', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 
                                      'arr_time', 'cancelled', 'cancellation_code', 'diverted', 'actual_elapsed_time', 'air_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime',
                                           'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_city_name', 'dest_city_name', 'dup', 'flights'])

# Orignal columns of df_flights_test:
# ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance']

df_flights_test_final = df_flights_test.drop(columns=['branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_city_name', 'dest_city_name', 'dup', 'flights'])
df_flights_test_final = df_flights_test_final.dropna()   # Drop rows with missing tail numbers (235 rows, or 0.14%)

In [45]:
df_flights_final;

In [46]:
df_flights_test_final;

In [16]:
# Export DFs to csv
df_flights_final.to_csv('df_flights_final.csv')
df_flights_test_final.to_csv('df_flights_test_final.csv')

# Hypothesis

I think these will be the most important feature(s):
* mkt and op carrier delay stats (mean/std)
* daily and hourly delay stats
* weather delay stats

I think these will be the least important feature(s):
* tail number stats